## Dependencias

In [ ]:
# Instalar dependencias
!pip install -q roboflow pycocotools segmentation-models-pytorch albumentations==1.4.7 torchmetrics==1.4.0.post0 onnx onnxruntime matplotlib seaborn scikit-learn

# Dataset
from roboflow import Roboflow
from pycocotools.coco import COCO
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import albumentations as A

# Pytorch
import torch
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.nn.functional as F

# Básicas
import random
import numpy as np
import matplotlib.pyplot as plt
import json

# Métricas
from torchmetrics import JaccardIndex
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score
import seaborn as sns

# Entrenamiento
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from tqdm.auto import tqdm
import time, copy

# Augmentations
import cv2

# Métricas
from torchmetrics import JaccardIndex
# 💡 AGREGADO: Importaciones para curvas PR y de Calibración
from sklearn.metrics import confusion_matrix, recall_score, precision_recall_curve, auc
from sklearn.calibration import calibration_curve
import seaborn as sns
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13

In [ ]:
# Comprobar GPU
print("GPU disponible:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# Reproducibilidad
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset

### Descargar Dataset de Roboflow

In [ ]:
ROBOFLOW_API_KEY = "x8f94bsxo45XbBc5XzuI"
WORKSPACE_NAME = "integrador-i"
PROJECT_NAME = "2d-materials-segmentation-zrowi"
VERSION_NUMBER = 2
DATASET_FORMAT = "coco-segmentation"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)
dataset = project.version(VERSION_NUMBER).download(DATASET_FORMAT)

print("\nDataset descargado en:", dataset.location)

### Augmentations

In [ ]:
IMAGE_SIZE = 512

train_transform = A.Compose([
    # Redimensionar y rellenar
    A.LongestMaxSize(max_size=IMAGE_SIZE, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),

    # Transformaciones geométricas suaves
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.05, rotate_limit=5, p=0.5),

    # Augmentations suaves específicas para microscopía
    A.RandomBrightnessContrast(p=0.7, brightness_limit=0.25, contrast_limit=0.25),
    A.HueSaturationValue(p=0.3, hue_shift_limit=5, sat_shift_limit=15, val_shift_limit=15),

    # Ruido/blur ligero para microscopía
    A.GaussianBlur(p=0.1, blur_limit=(1,3)),
    A.GaussNoise(p=0.2, var_limit=(5, 15)),

    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

eval_transform = A.Compose([
    A.LongestMaxSize(max_size=IMAGE_SIZE, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

### Creación de los Datasets

In [ ]:
def coco_image_info(coco_obj):
    for img_id in coco_obj.getImgIds():
        yield coco_obj.loadImgs([img_id])[0]

def build_mask(coco_obj, image_info):
    # Crea máscara de tamaño (H,W) con enteros en [0, num_classes-1]
    anns_ids = coco_obj.getAnnIds(imgIds=[image_info["id"]])
    anns = coco_obj.loadAnns(anns_ids)
    mask = np.zeros((image_info["height"], image_info["width"]), dtype=np.uint8)

    for ann in anns:
        original_cat_id = ann["category_id"]
        if original_cat_id in cat_id_to_index:
            cat_idx = cat_id_to_index[original_cat_id]
            rle = coco_obj.annToRLE(ann)
            m = coco_obj.annToMask(ann)  # (H,W) binaria
            mask[m == 1] = cat_idx

    return mask

data_dir = Path(dataset.location) # Ubicación del Dataset
SPLITS = ["train", "valid", "test"]
ANNOTATION_FILENAME = "_annotations.coco.json"

# Cargar annotations de COCO por split
coco_dict = {}
for split in SPLITS:
    ann_path = data_dir / split / ANNOTATION_FILENAME
    coco_dict[split] = COCO(str(ann_path))

# Mapeo: category_id -> 0..C-1
categories = coco_dict["train"].loadCats(coco_dict["train"].getCatIds())
categories[0]['name'] = 'background'

# Mapeo: category_id -> índice secuencial
cat_id_to_index = {}
index_to_name = {}
for i, cat in enumerate(categories):
    cat_id_to_index[cat["id"]] = i
    index_to_name[i] = cat["name"]

num_classes = len(categories)
print("Clases:", index_to_name)

In [ ]:
class GrapheneCocoDataset(Dataset):
    def __init__(self, coco_obj, split, transform):
        self.coco = coco_obj
        self.split = split
        self.transform = transform
        self.images = list(coco_image_info(coco_obj))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        info = self.images[idx]
        img_path = data_dir / self.split / info["file_name"]
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = build_mask(self.coco, info)

        # Albumentations espera dict
        transformed = self.transform(image=image, mask=mask)
        image_t = transformed["image"]
        mask_t = transformed["mask"]

        # A -> Tensor manual
        image_t = torch.from_numpy(image_t.transpose(2,0,1)).float()
        mask_t = torch.from_numpy(mask_t).long()
        return image_t, mask_t, info["file_name"]

train_ds = GrapheneCocoDataset(coco_dict["train"], "train", train_transform)
val_ds   = GrapheneCocoDataset(coco_dict["valid"], "valid", eval_transform)
test_ds  = GrapheneCocoDataset(coco_dict["test"], "test", eval_transform)

print("Tamaños:", len(train_ds), len(val_ds), len(test_ds))

### DataLoaders

In [ ]:
BATCH_SIZE = 4  # Reducido para estabilidad
NUM_WORKERS = 0  # Deshabilitado multiprocessing

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=False,
    drop_last=False
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=False,
    drop_last=False
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=False,
    drop_last=False
)

print("Batches train:", len(train_loader))
print("Batch size:", BATCH_SIZE)

## Modelo

In [ ]:
# ============== MODELO UNET++ ==============
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modelo: U-Net++ con encoder Efficientnet-B4 pretrained en ImageNet
model = smp.Unet(
    encoder_name="efficientnet-b5", # Para imágenes de microscopía
    encoder_weights="imagenet", # (INVESTIGAR)
    in_channels=3,
    classes=num_classes
).to(DEVICE)


### Funciones Loss

In [ ]:
class BalancedFocalLoss(nn.Module):
    """
    Implementación optimizada de Focal Loss balanceado para segmentación científica.
    Especialmente diseñado para manejar clases minoritarias como few-layer graphene.
    """
    def __init__(self, alpha=None, gamma=2.5, smooth=1e-8, reduction='mean'):
        super().__init__()
        self.alpha = alpha  # Pesos por clase para balancear
        self.gamma = gamma  # Valor 2.5 para enfoque en ejemplos difíciles
        self.smooth = smooth
        self.reduction = reduction

    def forward(self, logits, targets):
        """
        Args:
            logits: (B, C, H, W) - salidas del modelo
            targets: (B, H, W) - ground truth labels
        """
        # Convertir a probabilidades con temperature scaling
        probs = F.softmax(logits / 1.2, dim=1)  # Temperatura 1.2 para mejor calibración

        # One-hot encoding de targets
        targets_one_hot = F.one_hot(targets, num_classes=logits.shape[1])
        targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()

        # Cross entropy component con label smoothing ligero
        ce_loss = F.cross_entropy(logits, targets, reduction='none', label_smoothing=0.05)

        # Probabilidad de la clase correcta para cada pixel
        pt = torch.sum(probs * targets_one_hot, dim=1)  # (B, H, W)
        pt = torch.clamp(pt, self.smooth, 1.0 - self.smooth)  # Evitar log(0)

        # Factor de modulación focal: (1-pt)^gamma
        focal_weight = (1 - pt) ** self.gamma

        # Aplicar pesos de clase si se proporcionan
        if self.alpha is not None:
            alpha_t = torch.zeros_like(targets, dtype=torch.float32, device=targets.device)
            for i, alpha_val in enumerate(self.alpha):
                alpha_t[targets == i] = alpha_val
            focal_loss = alpha_t * focal_weight * ce_loss
        else:
            focal_loss = focal_weight * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class DiceLoss(nn.Module):
    """Dice Loss mejorado con suavizado adaptativo"""
    def __init__(self, smooth=1.0, ignore_index=None):
        super().__init__()
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, logits, targets):
        # logits: (B,C,H,W); targets: (B,H,W)
        probs = torch.softmax(logits, dim=1)
        targets_onehot = torch.nn.functional.one_hot(targets, num_classes=probs.shape[1]).permute(0,3,1,2).float()

        if self.ignore_index is not None:
            mask = targets != self.ignore_index
            mask = mask.unsqueeze(1)
            probs = probs * mask
            targets_onehot = targets_onehot * mask

        dims = (0,2,3)
        intersection = (probs * targets_onehot).sum(dims)
        cardinality = probs.sum(dims) + targets_onehot.sum(dims)
        dice = (2.0 * intersection + self.smooth) / (cardinality + self.smooth)
        return 1 - dice.mean()

def compute_class_weights(dataset, num_classes):
    # Calculando pesos de clase
    counts = torch.zeros(num_classes)
    for i in range(len(dataset)):
        _, mask, _ = dataset[i]
        for c in range(num_classes):
            counts[c] += (mask == c).sum().item()
    freq = counts / counts.sum()
    class_weights = (1.0 / np.sqrt(freq))
    class_weights = class_weights / class_weights.sum() * len(class_weights)  # normalize

    return class_weights

class TverskyLoss(nn.Module):
    """
    Tversky Loss - especialmente efectivo para clases desbalanceadas
    Alpha controla False Negatives, Beta controla False Positives
    """
    def __init__(self, alpha=0.7, beta=0.3, smooth=1.0):
        super().__init__()
        self.alpha = alpha  # Peso para FN (mayor = penaliza más missed detections)
        self.beta = beta    # Peso para FP (menor = menos penalización por false alarms)
        self.smooth = smooth

    def forward(self, logits, targets):
        probs = F.softmax(logits, dim=1)
        targets_onehot = F.one_hot(targets, num_classes=probs.shape[1]).permute(0,3,1,2).float()

        dims = (0,2,3)
        intersection = (probs * targets_onehot).sum(dims)
        fps = ((1 - targets_onehot) * probs).sum(dims)
        fns = (targets_onehot * (1 - probs)).sum(dims)

        tversky = (intersection + self.smooth) / (intersection + self.alpha * fns + self.beta * fps + self.smooth)
        return 1 - tversky.mean()

# Computar pesos de clase optimizados
class_weights = compute_class_weights(train_ds, num_classes)
class_weights[2] *= 50

# Instanciar funciones de pérdida mejoradas
focal_loss = BalancedFocalLoss(alpha=torch.tensor(class_weights).float().to(DEVICE), gamma=2.0)
dice_loss = DiceLoss(smooth=1.0)
tversky_loss = TverskyLoss(alpha=0.7, beta=0.3)  # Favorece recall sobre precision

def combined_loss_v3(logits, targets):
    """
    Función de pérdida tri-híbrida optimizada para few-layer detection:
    - 50% Focal Loss balanceado (para clases desbalanceadas)
    - 30% Tversky Loss (para mejorar recall de clases minoritarias)
    - 20% Dice Loss (para preservar forma de objetos)
    """
    focal = focal_loss(logits, targets)
    tversky = tversky_loss(logits, targets)
    dice = dice_loss(logits, targets)
    return 0.5 * focal + 0.3 * tversky + 0.2 * dice

### Métricas

In [ ]:
def calculate_iou_scores(outputs, targets, num_classes):
    iou = JaccardIndex(num_classes=num_classes, average='macro', task='multiclass').to(DEVICE)

    # Calcular mIoU
    iou.update(outputs, targets)
    iou_score = iou.compute()

    return iou_score.item()

def evaluate_eval(val_loader, model, loss_fn, num_classes=2):
    """
    Evaluates model on val_loader and returns loss, mIoU, and recall.
    Returns: loss, miou, recall
    """
    model.eval()

    losses, mious, recalls = [], [], []

    with torch.no_grad():
        for images, masks, *_ in val_loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            # Forward pass
            logits = model(images)
            loss = combined_loss_v3(logits, masks)

            # Predictions
            preds = torch.argmax(logits, dim=1)

            # IoU (mean over classes, MACRO-promedio)
            miou = calculate_iou_scores(preds, masks, num_classes)

            # Recall (MACRO-promedio)
            recall = recall_score(masks.cpu().numpy().ravel(), preds.cpu().numpy().ravel(), average="macro")

            losses.append(loss.item())
            mious.append(miou)
            recalls.append(recall)

    return np.mean(losses), np.mean(mious), np.mean(recalls)

def evaluate_train(images, masks, model, loss_fn, num_classes):
    images, masks = images.to(DEVICE), masks.to(DEVICE)

    with torch.amp.autocast("cuda"):
        logits = model(images)
        loss = loss_fn(logits, masks)
    preds = torch.argmax(logits, dim=1)

    miou = calculate_iou_scores(preds, masks, num_classes)
    # 💡 NOTA: Se asume que 'accuracy_score' está importada de sklearn
    # accuracy = accuracy_score(masks.cpu().numpy().ravel(), preds.cpu().numpy().ravel())
    # Tu codigo original tenía un error de sintaxis en accuracy, lo dejaremos como estaba
    # o mejor: lo arreglaremos si usas sklearn.metrics.accuracy_score
    recall = recall_score(masks.cpu().numpy().ravel(), preds.cpu().numpy().ravel(), average="macro")

    return loss, miou, recall


def evaluate_test_final(test_loader, model, num_classes):
    """
    Evalúa el modelo en el conjunto de prueba y recolecta datos para curvas PR y de Calibración.
    """
    model.eval()
    all_probas = []
    all_targets = []

    with torch.no_grad():
        for images, masks, *_ in tqdm(test_loader, desc="Evaluando para Curvas"):
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            # Forward pass
            logits = model(images)

            # 💡 Probas calibradas (con temperature scaling 1.2, como en BalancedFocalLoss)
            probas = F.softmax(logits / 1.2, dim=1)

            # Aplanar Probas (B, C, H, W) -> (N_pixeles, C)
            all_probas.append(probas.permute(0, 2, 3, 1).reshape(-1, num_classes).cpu().numpy())

            # Aplanar Targets (B, H, W) -> (N_pixeles,)
            all_targets.append(masks.flatten().cpu().numpy())

    all_probas_flat = np.concatenate(all_probas, axis=0)
    all_targets_flat = np.concatenate(all_targets, axis=0)

    return all_probas_flat, all_targets_flat


### Resumen del Modelo

In [ ]:
print("Modelo optimizado para few-layer detection configurado")
print(f"Pesos de clase aplicados: {[f'{w:.3f}' for w in class_weights]}")
print("Loss function: 50% Focal + 30% Tversky + 20% Dice")

## Entrenamiento

### Hiperparámetros, Optimizer y Scheduler

In [ ]:
EPOCHS = 120   # Más épocas para convergencia con loss compleja
LR = 6e-4     # Learning rate ligeramente menor para estabilidad
WARMUP = 0.2  # 20% para warmup
PATIENCE = 25 # Paciencia de épocas sin mejora

# Optimizer con grupos de parámetros diferenciados
optimizer = AdamW(model.parameters(), lr=LR,
                  weight_decay=0.01, betas=(0.9, 0.999), eps=1e-8)

total_steps = len(train_loader) * EPOCHS
scheduler = OneCycleLR(optimizer, max_lr=6e-4, # LR máximo conservador
                      total_steps=total_steps, pct_start=WARMUP, # Tiempo para warm-up
                      anneal_strategy='cos', # Cosine annealing suave
                      div_factor=10, # LR inicial = max_lr/10
                      final_div_factor=100 # LR final muy bajo
                      )

scaler = torch.amp.GradScaler('cuda', enabled=True)


### Loop de Entrenamiento

In [ ]:
HISTORY_FILE = "training_history.json"

with open(HISTORY_FILE, 'r') as f:
    history = json.load(f)

start_epoch = len(history["train_loss"]) + 1
best_miou = max(history["val_miou"]) if history["val_miou"] else 0.0

best_weights = None
bad_epochs = 0

In [ ]:
for epoch in range(1, EPOCHS + 1):
    current_lr = optimizer.param_groups[0]['lr']
    history["learning_rates"].append(current_lr)

    # Entrenamiento
    model.train()
    train_losses, train_ious, train_recalls = [], [], []
    for images, masks, _ in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad(set_to_none=True)

        loss, miou, recall = evaluate_train(images, masks, model, combined_loss_v3, num_classes)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_losses.append(loss.item())
        train_ious.append(miou)
        train_recalls.append(recall)

    # Métricas de entrenamiento
    history["train_loss"].append(np.mean(train_losses))
    history["train_miou"].append(np.mean(train_ious))
    history["train_recall"].append(np.mean(train_recalls))

    # Validación
    val_loss, val_miou, val_recall = evaluate_eval(val_loader, model, combined_loss_v3, num_classes)

    history["val_miou"].append(val_miou)
    history["val_loss"].append(val_loss)
    history["val_recall"].append(val_recall)

    print(f"[Epoch {epoch:2d}] TrainLoss={history['train_loss'][-1]:.4f}  ValLoss={val_loss:.4f}  ValmIoU={val_miou:.4f}")

    # Early stopping
    if val_miou > best_miou:
        best_miou = val_miou
        best_weights = copy.deepcopy(model.state_dict())
        bad_epochs = 0
        torch.save({
            'model_state_dict': best_weights,
            'epoch': epoch,
            'miou': best_miou,
            'class_weights': class_weights,
            'index_to_name': index_to_name
        }, "best_model.pt")
        print(f"✓ Modelo mejorado - mIoU: {best_miou:.4f}")
    else:
        bad_epochs += 1
        if bad_epochs >= PATIENCE:
            print(f"Early stopping en época {epoch} (sin mejora por {PATIENCE} épocas)")
            break

print("\nENTRENAMIENTO COMPLETADO")
print(f"Mejor mIoU validación: {best_miou:.4f}")
model.load_state_dict(best_weights)

### Visualización del Entrenamiento

In [ ]:
def med_des(array, txt = ''):
  print(f"Media {txt}: {np.mean(array[20:])}\nDesviación estándar {txt}:{np.std(array[20:])}\n")

In [ ]:
# Figura 1: Loss
print('Loss')
med_des(history["train_loss"], 'train')
med_des(history["val_loss"], 'val')

plt.figure(figsize=(6, 4))
plt.plot(history["train_loss"], label="Train")
plt.plot(history["val_loss"], label="Val")
plt.title("Perdida durante el Entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Valor de Pérdida Híbrida Balanceada")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Figura 2: mIoU
print('MIou')
med_des(history["train_miou"], 'train')
med_des(history["val_miou"], 'val')

plt.figure(figsize=(6, 4))
plt.plot(history["train_miou"], label="Train")
plt.plot(history["val_miou"], label="Val")
plt.title("mIoU (Mean Intersection over Union)")
plt.xlabel("Épocas")
plt.ylabel("Valor de mIoU")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Figura 3: Recall
print('Recall')

med_des(history["train_recall"], 'train')
med_des(history["val_recall"], 'val')

plt.figure(figsize=(6, 4))
plt.plot(history["train_recall"], label="Train")
plt.plot(history["val_recall"], label="Val")
plt.title("Recall (Sensibilidad)")
plt.xlabel("Épocas")
plt.ylabel("Valor de Recall")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Figura 4: Matriz de Confusión
class_names = ["background", "bulk", "few-layer"]
num_classes = len(class_names)

# --- Evaluación sobre el test set ---
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for images, masks, *_ in test_loader:
        images, masks = images.to(DEVICE), masks.to(DEVICE)

        with torch.cuda.amp.autocast():
            logits = model(images)  # [B, C, H, W]

        preds = torch.argmax(logits, dim=1)  # [B, H, W]

        all_preds.append(preds.cpu().numpy().ravel())
        all_targets.append(masks.cpu().numpy().ravel())

# Concatenar todos los píxeles
all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

# --- Calcular matriz de confusión ---
cm = confusion_matrix(all_targets, all_preds, labels=list(range(num_classes)))
cm_norm = cm.astype("float") / (cm.sum(axis=1)[:, np.newaxis] + 1e-9)  # evita división por 0
cm_norm = cm_norm * 100  # convertir a porcentaje

plt.figure(figsize=(6, 5))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Clase Predicha (Predicted)")
plt.ylabel("Clase Verdadera (True)")
plt.title("Matriz de Confusión (% por Fila)")
plt.show()



> **TO DO:** Preprocesamiento (filtrado EJ: Gaussiano) de las imagenes para few-layer, aumentar número de epochs, Gráfica de Precisión-Recall, Ajuste hiperparámetros en calibración.



In [ ]:
def plot_precision_recall_curve(probas, targets, num_classes, index_to_name):
    targets_one_hot = np.eye(num_classes)[targets]

    for i in range(num_classes):
        precision, recall, _ = precision_recall_curve(targets_one_hot[:, i], probas[:, i])
        pr_auc = auc(recall, precision)
        plt.plot(recall, precision, lw=2,
                 label=f'{index_to_name[i]} (AUC-PR = {pr_auc:.3f})')
    plt.ylabel("Precision (Sensibilidad)")
    plt.xlabel("Recall (Precisión)")
    plt.title("Curva de Precisión-Recall por Clase")
    plt.legend()
    plt.tight_layout()
    plt.show()


def plot_calibration_curve(probas, targets, num_classes, index_to_name, n_bins=50):
    max_probas = np.max(probas, axis=1)
    preds = np.argmax(probas, axis=1)

    is_correct = (preds == targets).astype(float)

    frac_of_pos, mean_predicted_value = calibration_curve(
        is_correct, max_probas, n_bins=n_bins, strategy='uniform'
    )

    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectamente Calibrado', color='gray')
    plt.plot(mean_predicted_value, frac_of_pos, label=f'Modelo (General)', lw=2)

    plt.xlabel("Confianza Predicha")
    plt.ylabel("Precisión Real")
    plt.title("Curva de Calibración del Modelo")
    plt.legend()
    plt.tight_layout()
    plt.show()

all_probas_flat, all_targets_flat = evaluate_test_final(
    test_loader,
    model,
    num_classes
)

plot_precision_recall_curve(
    all_probas_flat,
    all_targets_flat,
    num_classes,
    index_to_name
)

plot_calibration_curve(
    all_probas_flat,
    all_targets_flat,
    num_classes,
    index_to_name
)